# **Confusable Expressions**

In [1]:
import gzip
import random
from collections import Counter

import numpy as np
import pytest
from cytoolz import concat, sliding_window

%precision 2

try:
    get_ipython()

    import ipytest

    ipytest.autoconfig()

    def init_test():
        ipytest.clean()

    def run_test():
        ipytest.run()

except NameError:

    def init_test():
        pass

    def run_test():
        pass

## Background

In [2]:
data = [line.split() for line in gzip.open('../data/confuse.txt.gz','rt')]
random.shuffle(data)

sentences_train = data[:50000]
sentences_test = data[50000:55000]

conf_set = ["their", "there", "they're"]

We want to try several similar, but slightly different approaches. Let's start by defining a general `Confuser` class that contains the logic for testing a method but doesn't instantiate any particular method. For extra reference, [this](http://introtopython.org/classes.html) might help.

In [3]:
class Confuser:
    def __init__(self, confusable_set):
        self.confusable_set = confusable_set

    def preprocess(self, sentence):
        return sentence

    def train(self, training_corpus):
        pass

    def guess(self, sentence, position):
        raise NotImplementedError

    def eval(self, test_corpus):
        """Run a confusable-guesser over a test corpus and report the accuracy."""
        total = 0
        right = 0
        for sentence in test_corpus:
            sentence = self.preprocess(sentence)
            for i, w in enumerate(sentence):
                if w in self.confusable_set:
                    guess = self.guess(sentence, i)
                    if guess == sentence[i]:
                        right += 1
                    total += 1
        return right / total * 100.0

### Random Guessing

If we know nothing at all about the problem other than the fact that we have three confusable words, we can randomly guess.

In [4]:
class RandomConfuser(Confuser):
    def guess(self, line, i):
        """Choose one of the confusable set at random"""
        return random.choice(self.confusable_set)

If there are three choices, then we'd expect to guess correctly about 33% of the time.

In [5]:
random_confuser = RandomConfuser(conf_set)
random_confuser.eval(sentences_test)

35.63

### Default Confuser

We can definitely do better than random guessing. One piece of information we can get from our training corpus is the relative frequencies of *there, they're,* and *their*. If they aren't all equally common (which they aren't), we can beat random guessing by always going with the most frequent option.

This might be inefficient and the worst method possible, but it often gives surprisingly good results. This is in part due to Zipf's Law, which tells us that there are a few very frequent things and lots of infrequent things. In other words, the frequencies of confusable items can be very uneven, and the more uneven the probabilities the better we do with a default choice.

Since this method is simple, we can use it to establish a *baseline* score that tells us how well we do if we do almost nothing. This gives a sense of how difficult a problem is. Any fancier solutions can be evaluated against the baseline.

In [6]:
class DefaultConfuser(Confuser):

    def preprocess(self, sentence):
        """Lowercase sentence."""
        return [w.lower() for w in sentence]

    def train(self, train_corpus):
        """Find the most common member of the confusion set in the training corpus."""
        train_corpus = [self.preprocess(sentence) for sentence in train_corpus]
        target_counts = Counter(
            w for w in concat(train_corpus) if w in self.confusable_set
        )
        self.default_guess = target_counts.most_common()[0][0]

    def guess(self, line, i):
        """Always guess the most frequent member of the confusion set."""
        return self.default_guess

In this particular case, going with a default is better than random guessing, but not by all that much.

In [7]:
baseline = DefaultConfuser(conf_set)
baseline.train(sentences_train)
baseline.eval(sentences_test)

50.39

### Bigram Model

With the baseline in mind, we can try using a **Bigram Language Model** to solve the confusables problem. Let's say we see the sequence *in their house* and we want to know if that's correct. We can use a language model to compare $P(\textit{in, their, house})$ to $P(\textit{in, there, house})$ and $P(\textit{in, they're, house})$. Whichever sequence has the highest probability is the one with the correct member of the confusion set (we hope!).

To start, by the chain rule we have:

$$P(\textit{in, their, house})=P(\textit{in})\times P(\textit{their}|\textit{in})\times P(\textit{house}|\textit{their,in})$$

We can simplify this two ways. First, since all the sequences we're testing start with *in*, we can leave out the $P(in)$ term without changing the relative order of the 3 sequences. Second, if we assume a first order Markov Model (=bigrams),  we only need to consider one word of context.

These two changes get us:

$$\begin{align}
P(\textit{their, house}|\textit{in})&=P(\textit{their}|\textit{in})\times P(\textit{house}|\textit{their})\\
&=\frac{C_B(\textit{in their})}{C_U(\textit{in})}\times\frac{C_B(\textit{their house})}{C_U(\textit{their})}
\end{align}$$

where $C_B$ and $C_U$ are bigram and unigram frequencies in the training corpus. We repeat this for *there* and *they're* and use the one with the highest probability. If the probability of all three options turns out to be zero, then we'll fall back on the most frequent overall choice, as we did for the default confuser. 


In [8]:
class BigramConfuser(DefaultConfuser):
    def preprocess(self, sentence):
        """Normalize sentence and add filler tokens <s> and </s>"""
        return ["<s>"] + [w.lower() for w in sentence] + ["</s>"]

    def get_unigram_counts(self, train_corpus):
        self.unigrams = Counter(concat(train_corpus))

    def get_bigram_counts(self, train_corpus):
        self.bigrams = Counter(sliding_window(2, concat(train_corpus)))

    def get_default_guess(self):
        self.default_guess = max((self.unigrams[w], w) for w in self.confusable_set)[1]

    def train(self, train_corpus):
        """Count bigram and unigram frequencies in the training corpus."""
        train_corpus = [self.preprocess(sentence) for sentence in train_corpus]
        self.get_unigram_counts(train_corpus)
        self.get_bigram_counts(train_corpus)
        self.get_default_guess()

    def guess(self, sentence, i):
        """Find the guess that maximizes the probability of the sequence"""
        best_p = 0.0
        best_guess = self.default_guess
        for guess in self.confusable_set:
            p = self.prob(guess, sentence, i)
            if p > best_p:
                best_p = p
                best_guess = guess
        return best_guess

    def prob(self, guess, sentence, i):
        """Calculate the probability of a guess in context."""
        try:
            p1 = self.bigrams[sentence[i - 1], guess] / self.unigrams[sentence[i - 1]]
            p2 = self.bigrams[guess, sentence[i + 1]] / self.unigrams[guess]
            return p1 * p2
        except ZeroDivisionError:
            return 0.0

The bigram model performs much better than the baseline:

In [9]:
bigram_confuser = BigramConfuser(conf_set)
bigram_confuser.train(sentences_train)
bigram_confuser.eval(sentences_test)

71.21

The **Error Reduction** is a measure of how much better one model is than another. In this case, the bigram model reduces the error rate of the default model by about 60%.

In [10]:
baseline_error = 1 - baseline.eval(sentences_test)
bigram_error = 1 - bigram_confuser.eval(sentences_test)
(bigram_error - baseline_error) / baseline_error * 100

42.15

One difficulty with implementing language models is that it's hard to know what the "right" answer is. A small error in our code might give us a lower accuracy, but how would we know? For testing purposes, it can be helpful to create tiny training and test sets that are small enough that we can do the relevant calculations by hand and then confirm that our code comes up with the same answer. 

In [11]:
%%ipytest

DATA = [
    "The jury did not elaborate , but it added that `` there should be periodic surveillance of the pricing practices of the concessionaires for the purpose of keeping the prices reasonable '' .",
    "Despite the warning , there was a unanimous vote to enter a candidate , according to Republicans who attended .",
    "When the crowd was asked whether it wanted to wait one more term to make the race , it voted no -- and there were no dissents .",
    "A highway department source said there also is a plan there to issue some $3 million to $4 million worth of rural roads authority bonds for rural road construction work .",
    "A highway department source said there also is a plan there to issue some $3 million to $4 million worth of rural roads authority bonds for rural road construction work .",
    "Pelham said Sunday night there was research being done on whether the `` quickie '' vote on the increase can be repealed outright or whether notice would have to first be given that reconsideration of the action would be sought .",
]


@pytest.fixture
def my_bigram_confuser():
    conf = BigramConfuser(["their", "there", "they're"])
    conf.train(s.split() for s in DATA)
    return conf


def test_bigram_confuser_default(my_bigram_confuser):
    assert my_bigram_confuser.default_guess == "there"


@pytest.mark.parametrize(
    "gram,count", [("<s>", 6), ("the", 11), ("to", 9), ("pelham", 1), ("zippy", 0)]
)
def test_bigram_confuser_uni(my_bigram_confuser, gram, count):
    assert my_bigram_confuser.unigrams[gram] == count


@pytest.mark.parametrize(
    "gram,count",
    [
        (("<s>", "<s>"), 0),
        (("</s>", "<s>"), 5),
        (("asked", "whether"), 1),
        (("the", "pinhead"), 0),
    ],
)
def test_bigram_confuser_bi(my_bigram_confuser, gram, count):
    assert my_bigram_confuser.bigrams[gram] == count


@pytest.mark.parametrize(
    "guess,seq,p",
    [
        ("there", ("said", "X", "also"), 0.1667),
        ("there", ("said", "X", "should"), 0.0833),
        ("their", ("said", "X", "should"), 0.0),
        ("their", ("before", "X", "after"), 0.0),
    ],
)
def test_bigram_confuser_prob(my_bigram_confuser, guess, seq, p):
    assert my_bigram_confuser.prob(guess, seq, 1) == pytest.approx(p, rel=1e-3)

..............                                                                               [100%]
14 passed in 0.15s


## Add-One Smoothing

We can improve our bigram model by smoothing, and the simplest method for smoothing is to simply add one to every count, giving us:

$$
P(\textit{their, house}|\textit{in})=\frac{C_B(\textit{in their})+1}{C_U(\textit{in})+V}\times\frac{C_B(\textit{their house})+1}{C_U(\textit{their})+V}
$$

where $V$ is the size of the vocabulary.

In [12]:
class BigramAddOneConfuser(BigramConfuser):
    def prob(self, guess, sentence, i):
        # Size of vocabulary
        V = len(self.unigrams)

        # Calculate smoothed probability
        bigram_p1_num = self.bigrams[(sentence[i - 1], guess)] + 1
        bigram_p1_den = self.unigrams[sentence[i - 1]] + V
        p1 = bigram_p1_num / bigram_p1_den
        
        # Calculate smoothed probability
        bigram_p2_num = self.bigrams[(guess, sentence[i + 1])] + 1
        bigram_p2_den = self.unigrams[guess] + V
        p2 = bigram_p2_num / bigram_p2_den
        
        # Return product
        return p1 * p2

In [13]:
bigram1_confuser = BigramAddOneConfuser(conf_set)
bigram1_confuser.train(sentences_train)
bigram1_confuser.eval(sentences_test)

86.38

In [14]:
bigram1_error = 1 - bigram1_confuser.eval(sentences_test)
print(
    f"Error reduction vs. baseline model = {(bigram1_error - baseline_error) / baseline_error * 100:.2f}%"
)
print(
    f"Error reduction vs. bigram model   = {(bigram1_error - bigram_error) / bigram_error * 100:.2f}%"
)

Error reduction vs. baseline model = 72.87%
Error reduction vs. bigram model   = 21.62%


In [15]:
%%ipytest

@pytest.fixture
def my_bigram1_confuser():
    conf = BigramAddOneConfuser(["their", "there", "they're"])
    conf.train(s.split() for s in DATA)
    return conf

@pytest.mark.parametrize(
    "guess,seq,p",
    [
        ("there", ("said", "X", "also"), 0.0008090),
        ("there", ("said", "X", "should"), 0.0005393),
        ("their", ("said", "X", "should"), 9.7087e-5),
        ("their", ("before", "X", "after"), 0.0001),
    ],
)
def test_bigram1_confuser_prob(my_bigram1_confuser, guess, seq, p):
    assert my_bigram1_confuser.prob(guess, seq, 1) == pytest.approx(p, rel=1e-3)

....                                                                                         [100%]
4 passed in 0.03s


## Trigram Model

We can increase the context sensitivity of our model without making it too much more complicated by moving to a **trigram** model. Using the chain rule again, we have:

$$
P(\textit{live, in, their, house, today})=P(\textit{live})\times P(\textit{in}|\textit{live})\times 
P(\textit{their}|\textit{in, live})\times
P(\textit{house}|\textit{their, in})\times
P(\textit{today}|\textit{house, their})
$$

As before, we can simplify this to:

$$\begin{align}
P(\textit{their, house, today}|\textit{in,live})&=P(\textit{their}|\textit{in, live})\times
P(\textit{house}|\textit{their, in})\times
P(\textit{today}|\textit{house, their})\\
&=\frac{C_T(\textit{live in their})}{C_B(\textit{live in})}\times
\frac{C_T(\textit{in their house})}{C_B(\textit{in their})}\times
\frac{C_T(\textit{their house today})}{C_B(\textit{their house})}
\end{align}$$

where $C_T$ and $C_B$ are trigram and bigram frequencies.

Now we implement and evaluate a trigram-based confusable solver.

In [16]:
class TrigramConfuser(BigramConfuser):
    def preprocess(self, sentence):
        """Normalize sentence and add filler tokens <s> and </s>"""
        return ["<s>", "<s>"] + [w.lower() for w in sentence] + ["</s>", "</s>"]

    def get_trigram_counts(self, train_corpus):
        self.trigrams = Counter(sliding_window(3, concat(train_corpus)))

    def train(self, train_corpus):
        """Calculate trigram frequencies in training corpus"""
        train_corpus = [self.preprocess(sentence) for sentence in train_corpus]
        self.get_unigram_counts(train_corpus)
        self.get_bigram_counts(train_corpus)
        self.get_trigram_counts(train_corpus)
        self.get_default_guess()

    def prob(self, guess, sentence, i):
        """Calculate probability of a guess in trigram context"""
        try:
            # Calculate probability with previous two words and guess
            trigram_p1_num = self.trigrams[(sentence[i - 2], sentence[i - 1], guess)]
            trigram_p1_den = self.bigrams[(sentence[i - 2], sentence[i - 1])]
            p1 = trigram_p1_num / trigram_p1_den

            # Calculate probability with guess, the word before it, and the word after it
            trigram_p2_num = self.trigrams[(sentence[i - 1], guess, sentence[i + 1])]
            trigram_p2_den = self.bigrams[(sentence[i - 1], guess)]
            p2 = trigram_p2_num / trigram_p2_den
            
            # Calculate probability with guess and next two words
            trigram_p3_num = self.trigrams[(guess, sentence[i + 1], sentence[i + 2])]
            trigram_p3_den = self.bigrams[(guess, sentence[i + 1])]
            p3 = trigram_p3_num / trigram_p3_den
            
            # Return product
            return p1 * p2 * p3
        except ZeroDivisionError:
            return 0.0

Now, we evaluate our model. How does it compare to the baseline and bigram models?

In [17]:
trigram_confuser = TrigramConfuser(conf_set)
trigram_confuser.train(sentences_train)
trigram_confuser.eval(sentences_test)

50.78

In [18]:
%%ipytest

@pytest.fixture
def my_trigram_confuser():
    conf = TrigramConfuser(["their", "there", "they're"])
    conf.train(s.split() for s in DATA)
    return conf

def test_trigram_confuser_default(my_trigram_confuser):
    assert my_trigram_confuser.default_guess == "there"

@pytest.mark.parametrize(
    "gram,count",
    [
        (("<s>", "<s>"), 6),
        (("</s>", "<s>"), 5),
        (("asked", "whether"), 1),
        (("the", "pinhead"), 0),
    ],
)
def test_trigram_confuser_bi(my_trigram_confuser, gram, count):
    assert my_trigram_confuser.bigrams[gram] == count

@pytest.mark.parametrize(
    "gram,count",
    [
        (("<s>", "<s>", "</s>"), 0),
        (("</s>", "</s>", "<s>"), 5),
        (("asked", "whether", "it"), 1),
        (("zippy", "the", "pinhead"), 0),
    ],
)
def test_trigram_confuser_tri(my_trigram_confuser, gram, count):
    assert my_trigram_confuser.trigrams[gram] == count

@pytest.mark.parametrize(
    "guess,seq,p",
    [
        ("there", ("source", "said", "X", "also", "is"), 1.0),
        ("their", ("source", "said", "X", "also", "is"), 0.0),
        ("there", ("sunday", "night", "X", "was", "research"), 0.5),
        ("their", ("sunday", "night", "X", "was", "research"), 0.0),
    ],
)
def test_trigram_confuser_prob(my_trigram_confuser, guess, seq, p):
    assert my_trigram_confuser.prob(guess, seq, 2) == pytest.approx(p, rel=1e-3)

.............                                                                                [100%]
13 passed in 0.20s


## Trigrams + Add-One Smoothing

Finally, we implement and evaluate a trigram model using Add-One Smoothing.

In [19]:
class TrigramAddOneConfuser(TrigramConfuser):
    def prob(self, guess, sentence, i):
        V = len(self.unigrams)  # Vocabulary size
        
        # Calculate trigram probability with previous two words and guess
        trigram_p1_num = self.trigrams[(sentence[i - 2], sentence[i - 1], guess)] + 1
        trigram_p1_den = self.bigrams[(sentence[i - 2], sentence[i - 1])] + V
        p1 = trigram_p1_num / trigram_p1_den
        
        # Calculate trigram probability with guess, words before it, and word after it
        trigram_p2_num = self.trigrams[(sentence[i - 1], guess, sentence[i + 1])] + 1
        trigram_p2_den = self.bigrams[(sentence[i - 1], guess)] + V
        p2 = trigram_p2_num / trigram_p2_den
        
        # Calculate trigram probability with guess and next two words
        trigram_p3_num = self.trigrams[(guess, sentence[i + 1], sentence[i + 2])] + 1
        trigram_p3_den = self.bigrams[(guess, sentence[i + 1])] + V
        p3 = trigram_p3_num / trigram_p3_den

        # Return product
        return p1 * p2 * p3

In [20]:
trigram1_confuser = TrigramAddOneConfuser(conf_set)
trigram1_confuser.train(sentences_train)
trigram1_confuser.eval(sentences_test)

66.73

In [21]:
%%ipytest

@pytest.fixture
def my_trigram1_confuser():
    conf = TrigramAddOneConfuser(["their", "there", "they're"])
    conf.train(s.split() for s in DATA)
    return conf

@pytest.mark.parametrize(
    "guess,seq,p",
    [
        ("there", ("source", "said", "X", "also", "is"), 2.544e-5),
        ("their", ("source", "said", "X", "also", "is"), 9.803e-7),
        ("there", ("sunday", "night", "X", "was", "research"), 7.688e-6),
        ("their", ("sunday", "night", "X", "was", "research"), 9.900e-7),
    ],
)
def test_trigram1_confuser_prob(my_trigram1_confuser, guess, seq, p):
    assert my_trigram1_confuser.prob(guess, seq, 2) == pytest.approx(p, rel=1e-3)

....                                                                                         [100%]
4 passed in 0.03s
